In [1]:
import pandas as pd

In [2]:
base = pd.read_csv("Multiple Cause of Death, 1999-2020 Age Gender Year Race COD Socioec.txt", sep='\t', skipfooter=0)

In [4]:
base.head(3)

,Notes,Multiple Cause of death,Multiple Cause of death Code,Year,Year Code,Gender,Gender Code,Race,Race Code,Five-Year Age Groups,Five-Year Age Groups Code,Deaths,Population,Crude Rate,socioec quintile
0,NaN,Opium,T40.0,1999,1999,Female,F,American Indian or Alaska Native,1002-5,< 1 year,1,0,9033,Unreliable,1
1,NaN,Opium,T40.0,1999,1999,Female,F,American Indian or Alaska Native,1002-5,1-4 years,4-Jan,0,38452,Unreliable,1
2,NaN,Opium,T40.0,1999,1999,Female,F,American Indian or Alaska Native,1002-5,5-9 years,9-May,0,56191,Unreliable,1


In [6]:
base["socioec quintile"].value_counts()

socioec quintile
1    24288
2    24288
3    24288
4    24288
5    24288
Name: count, dtype: int64

In [15]:
base["Crude Rate"] = pd.to_numeric(base["Crude Rate"], errors='coerce')
base["Deaths"] = pd.to_numeric(base["Deaths"], errors='coerce')

In [16]:
base.groupby(["socioec quintile"])["Crude Rate"].sum()

socioec quintile
1    10448.716310
2     8547.068287
3    12041.539221
4     9225.550522
5    10015.898696
Name: Crude Rate, dtype: float64

In [17]:
base.groupby(["socioec quintile"])["Deaths"].sum()

socioec quintile
1    127505.0
2    126168.0
3    133706.0
4    146814.0
5    127087.0
Name: Deaths, dtype: float64